In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 69.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import evaluate

from edge_case_feedback import EdgeCaseFeedback

## ***Get Dataset - BigCodeBench***

In [ ]:
dataset = load_dataset("bigcode/bigcodebench", split="v0.1.0_hf[:10]")

## ***Initialize Models***

In [ ]:
model_names = {                               #TODO : Change Model to check improvement
    "CodeLlama": "codellama/CodeLlama-7b-hf",
}

tokenizers = {}
models = {}

In [ ]:
for name, model_id in model_names.items():
    tokenizers[name] = AutoTokenizer.from_pretrained(model_id)
    models[name] = AutoModelForCausalLM.from_pretrained(model_id)

## ***Feedback Model***

In [ ]:
def generate_feedback(prompt, generated_code, solution):
    edge_feedback = EdgeCaseFeedback(models["CodeLlama"])  # or whichever model you're using
    test_cases = edge_feedback.generate_test_cases(prompt, generated_code)
    results = edge_feedback.evaluate_test_cases(generated_code, test_cases)
    
    # Create enhanced prompt with edge case feedback
    enhanced_prompt = f"""
    Original prompt: {prompt}
    
    Consider these edge cases and their requirements:
    {results['feedback']}
    
    Please regenerate the code to handle these edge cases correctly.
    """
    
    return enhanced_prompt

In [1]:
def generate_code(prompt, model, tokenizer, solution, max_length=1000):
    n = 5  
    current_prompt = prompt
    best_score = 0
    best_code = None
    
    for i in range(n):
        inputs = tokenizer(current_prompt, return_tensors="pt")
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_p=0.9, do_sample=True)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Calculate BLEU score
        bleu_score = bleu_metric.compute(predictions=[generated_code], references=[solution])["bleu"]
        
        if bleu_score > best_score:
            best_score = bleu_score
            best_code = generated_code
        
        # Generate feedback for next iteration
        current_prompt = generate_feedback(prompt, generated_code, solution)
    
    return best_code

In [ ]:
generated_codes = {name: [] for name in model_names}

for example in dataset:
    print("Prompt:")
    prompt = example["complete_prompt"]
    solution = example["canonical_solution"]
    for model_name in model_names:
        generated_code = generate_code(prompt, models[model_name], tokenizers[model_name], solution)
        generated_codes[model_name].append(generated_code)

## ***Evaluation***

In [ ]:
# Evaluation Setup with BLEU (or CodeBLEU if available)
bleu_metric = evaluate.load("bleu")

# Prepare reference code for evaluation
references = [example["canonical_solution"] for example in dataset]

# Evaluate each model's generated code against the reference code
evaluation_scores = {}
for model_name, codes in generated_codes.items():
    bleu_score = bleu_metric.compute(predictions=codes, references=references)
    evaluation_scores[model_name] = bleu_score["bleu"]
    print(f"{model_name} BLEU Score:", bleu_score["bleu"])

# Print final evaluation summary
print("\n=== Evaluation Summary ===")
for model_name, score in evaluation_scores.items():
    print(f"{model_name} BLEU Score: {score:.4f}")